In [13]:
import json
import boto3
import sys
import time
import pandas as pd
import requests

In [14]:
query = 'SELECT hpaths."_remember_id", hpaths."data" FROM "reports_neo"."doc_type" as dtype left join "reports_neo"."hocr_paths" as hpaths on dtype."_remember_id" = hpaths."_remember_id"'
s3resource = boto3.resource("s3")
athena_client = boto3.client("athena")
ssm = boto3.client("ssm")
root_url = ssm.get_parameter(Name=f"/account/root-url")["Parameter"]["Value"]
apikey = ssm.get_parameter(Name="/account/internal-api-key")["Parameter"]["Value"]
base_url = f"https://tagentities.{root_url}"
acc_owner = ssm.get_parameter(Name="/account/owner")["Parameter"]["Value"].upper()
headers = {"x-api-key": apikey}


In [15]:
acc_owner

'USBANK'

In [16]:
def fetchall_athena(query_string, client, athena_bucket, get_first_col=False):
    query_id = client.start_query_execution(
        QueryString=query_string,
        QueryExecutionContext={"Database": "reports_neo"},
        ResultConfiguration={"OutputLocation": f"s3://{athena_bucket}"},
    )["QueryExecutionId"]
    query_status = None
    while query_status == "QUEUED" or query_status == "RUNNING" or query_status is None:
        query_status = client.get_query_execution(QueryExecutionId=query_id)[
            "QueryExecution"
        ]["Status"]["State"]
        if query_status == "FAILED" or query_status == "CANCELLED":
            raise Exception(
                'Athena query with the string "{}" failed or was cancelled'.format(
                    query_string
                )
            )
        time.sleep(10)
    results_paginator = client.get_paginator("get_query_results")
    results_iter = results_paginator.paginate(
        QueryExecutionId=query_id, PaginationConfig={"PageSize": 1000}
    )
    results = []
    data_list = []
    for results_page in results_iter:
        for row in results_page["ResultSet"]["Rows"]:
            data_list.append(row["Data"])
    if get_first_col:
        start_index = 0
    else:
        start_index = 1
    for datum in data_list[start_index:]:
        results.append([x.get("VarCharValue") for x in datum])
    return [tuple(x) for x in results]


def get_athena_bucket(s3resource):
    # Get athena bucket name
    all_buckets = list(s3resource.buckets.all())
    athena_buckets = [x.name for x in all_buckets if "athena" in x.name]
    if len(athena_buckets):
        return athena_buckets[0]
    print("No athena buckets available")
    sys.exit()


    
def rerun(rid, path):
    try:
        payload = {
            "_remember_id": rid,
            "_paginated_hocr_metadata_path": path,
            "_path": "tagging",
            "_read_wp_task_token": "fake",
            "AccountOwner": acc_owner,
            "taskToken": "fake"
        }
        rer = requests.post(
                url=base_url,
                headers=headers,
                data=json.dumps(payload),
                verify=True,
            )
        return json.loads(rer.text)['_execution_arn']
    except:
        return "failed"


In [17]:
athena_bucket = get_athena_bucket(s3resource)
hocr_paths = pd.DataFrame(fetchall_athena(query, athena_client, athena_bucket))

In [18]:
hocr_paths.columns = [ "_remember_id", "_paginated_hocr_metadata_path"]
hocr_paths=hocr_paths.dropna()

In [19]:
None in list(hocr_paths._remember_id.unique())

False

In [20]:
hocr_paths

,_remember_id,_paginated_hocr_metadata_path
4,2bedf777-f8ad-45f8-a2f4-33398948c6a2,S3://hw-app-usbank-training/aiva/SI1/2019:11:1...
21,86f2c031-d0f9-444e-a1f1-c03019b11cc6,S3://hw-app-usbank-training/aiva/SI/2020:07:15...
24,d2f0b5fa-18a2-46a5-80c2-df78f14c79bf,S3://hw-app-usbank-training/aiva/SI/2020:07:15...
38,a5584b82-e49b-4dd4-a9a7-b45ad3387ddf,S3://hw-app-usbank-training/aiva/SI1/2020:01:2...
40,a04340de-81dd-4b45-b725-5f5467401ff3,S3://hw-app-usbank-training/aiva/SI1/2020:01:2...
...,...,...
1333198,f0393e35-32cc-4455-a377-3ffe72130036,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...
1333204,e5e7b447-718a-48ed-b0cb-ce277f90acd7,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...
1333207,f86b8099-e11b-4f20-8449-68bfbabf04eb,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...
1333208,fb8083f3-c5fa-4275-82f7-b34dfbce9cd9,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...


In [ ]:
hocr_paths["c"] = hocr_paths.apply(lambda row: rerun(row["_remember_id"], row["_paginated_hocr_metadata_path"]), axis = 1)

In [ ]:
len(hocr_paths)

220272

In [ ]:
hocr_paths

,_remember_id,_paginated_hocr_metadata_path,c
4,2bedf777-f8ad-45f8-a2f4-33398948c6a2,S3://hw-app-usbank-training/aiva/SI1/2019:11:1...,arn:aws:states:us-east-1:727115606880:executio...
21,86f2c031-d0f9-444e-a1f1-c03019b11cc6,S3://hw-app-usbank-training/aiva/SI/2020:07:15...,arn:aws:states:us-east-1:727115606880:executio...
24,d2f0b5fa-18a2-46a5-80c2-df78f14c79bf,S3://hw-app-usbank-training/aiva/SI/2020:07:15...,arn:aws:states:us-east-1:727115606880:executio...
38,a5584b82-e49b-4dd4-a9a7-b45ad3387ddf,S3://hw-app-usbank-training/aiva/SI1/2020:01:2...,arn:aws:states:us-east-1:727115606880:executio...
40,a04340de-81dd-4b45-b725-5f5467401ff3,S3://hw-app-usbank-training/aiva/SI1/2020:01:2...,arn:aws:states:us-east-1:727115606880:executio...
...,...,...,...
1333198,f0393e35-32cc-4455-a377-3ffe72130036,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...,arn:aws:states:us-east-1:727115606880:executio...
1333204,e5e7b447-718a-48ed-b0cb-ce277f90acd7,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...,arn:aws:states:us-east-1:727115606880:executio...
1333207,f86b8099-e11b-4f20-8449-68bfbabf04eb,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...,arn:aws:states:us-east-1:727115606880:executio...
1333208,fb8083f3-c5fa-4275-82f7-b34dfbce9cd9,S3://hw-app-usbank-training/aiva/SI1/2020:02:1...,arn:aws:states:us-east-1:727115606880:executio...
